# Interacting with Azure OpenAI with GenAI Gateway Capabilities by APIM

NB: World Bank docimenation is [here](https://aiqa.worldbank.org/chat/api-platform/documentation?currentTab=desktop)

This Jupyter Notebook demonstrates how to connect to Azure OpenAI’s API through the World Bank’s APIM endpoint using Azure Active Directory for authentication. It includes generating a chat completion response based on a prompt related to World Bank projects.

##### Requirements

Before running this notebook, ensure that the following Python libraries are installed:
```bash
pip install openai azure-identity
```

---

### **Device Code Credential Authentication Sytem**

**Use Case:**  
This approach is for data scientists or developers running scripts locally to authenticate with Azure services. It is not intended for application development purposes.

**Example Scenario:**  
A data scientist running a script locally on their machine needs to connect to Azure OpenAI to assist with data processing. Since the script is running in an environment without a browser, the data scientist can use Device Code Credential to authenticate by entering a device code on a separate device with browser access. This method also allows the data scientist or developer to check out different models without needing to develop an application.


---




In [1]:
from openai import AzureOpenAI
from azure.identity import InteractiveBrowserCredential, get_bearer_token_provider, DeviceCodeCredential, InteractiveBrowserCredential
token_provider = get_bearer_token_provider(
    DeviceCodeCredential(tenant_id="31a2fec0-266b-4c67-b56e-2796d8f59c36", client_id ="00c104af-b0ae-4557-9787-6e6cfced741e"),
    "https://cognitiveservices.azure.com/.default")

ModuleNotFoundError: No module named 'pydantic_core._pydantic_core'

In [30]:
# initialize AzureOpenAI client
client = AzureOpenAI(
    azure_endpoint="https://azapimdev.worldbank.org/conversationalai/v2/",
    azure_ad_token_provider=token_provider,
    api_version="2024-12-01-preview"
)


In [32]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
# other models available: 'gpt-4o', 'gpt-35-turbo-16k', 'gpt-4'
# quick tips for data scientist: Use GPT 4o mini for POC due to latency and higher context window compared to GPT 3.5.
    messages=[
        {"role": "system", "content": "You are an AI assistant helping with World Bank projects."},
        {"role": "user", "content": "What are the three main kinds of macroeconomic simulation model use short bullets in your answer."}
        #{"role": "user", "content": "Explain the importance of financial inclusion in global development in 4 sentences."}
    ],
    # extra_headers={'api-key': APIM_SUBSCRIPTION_KEY} # API key is optional
)

# print the response
print(response.choices[0].message.content)

The three main kinds of macroeconomic simulation models are:

- **Static Models**: Analyze economic relationships at a single point in time, providing a snapshot of an economy's structure and equilibrium without accounting for dynamic changes over time.

- **Dynamic Models**: Incorporate time as a variable, allowing for the analysis of how economies evolve over time under various scenarios and policy changes.

- **Stochastic Models**: Include random variables and shocks, enabling the assessment of uncertain outcomes and the impact of unexpected events on the economy.


# This doesnot seem to trigger the requirement of a new approval

In [33]:
from langchain_openai import AzureOpenAIEmbeddings

In [34]:
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://azapimdev.worldbank.org/conversationalai/v2",
    azure_ad_token_provider=token_provider,
    # api_version="2024-05-01-preview",
    api_version="2024-10-01-preview",
    # model = "text-embedding-ada-002"
    model = "text-embedding-3-small"
)

In [35]:
embeddings.embed_documents("Hello, world!")

[[0.02750912494957447,
  0.0018673025770112872,
  0.015475830063223839,
  0.05215206742286682,
  0.01204887218773365,
  0.01716594398021698,
  -0.011192132718861103,
  0.008621914312243462,
  -0.009206054732203484,
  -0.06498758494853973,
  -0.0010008275276049972,
  0.009548750706017017,
  -0.08262083679437637,
  -0.008941244333982468,
  0.00985250435769558,
  0.01649612933397293,
  -0.015374579466879368,
  0.008458355441689491,
  0.005549335386604071,
  0.058725595474243164,
  -0.007508153095841408,
  0.015452465042471886,
  0.03950347751379013,
  0.026013726368546486,
  -0.0038494861219078302,
  3.370977719896473e-05,
  -0.00919047836214304,
  -0.009556539356708527,
  0.04994012415409088,
  -0.023163119331002235,
  0.05031397193670273,
  -0.02685488760471344,
  0.031870707869529724,
  -0.006737087853252888,
  -0.011301172897219658,
  0.02916029654443264,
  -0.022976195439696312,
  0.05096821114420891,
  -0.00039380809175781906,
  0.03732268512248993,
  -0.00798325426876545,
  0.01130

In [36]:
embeddings = embeddings.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

embeddings.embed_doc("Hello, world!")

In [37]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
# other models available: 'gpt-4o', 'gpt-35-turbo-16k', 'gpt-4'
# quick tips for data scientist: Use GPT 4o mini for POC due to latency and higher context window compared to GPT 3.5.
    messages=[
        {"role": "system", "content": "You are an AI assistant helping with World Bank projects."},
        {"role": "user", "content": "Explain in 3 sentences in what world bank department one can find macroeconomic modellers."}
    ],
    # extra_headers={'api-key': APIM_SUBSCRIPTION_KEY} # API key is optional
)

# print the response
print(response.choices[0].message.content)

Macroeconomic modellers can be found in the World Bank's Poverty and Equity Global Practice and the Macroeconomics, Trade, and Investment Global Practice. These departments focus on analyzing economic trends, developing macroeconomic models, and providing policy advice to enhance economic growth and reduce poverty. Their work often involves the assessment of macroeconomic stability, fiscal policies, and the impacts of global economic changes on developing countries.


# This on the other hand does

Get Bearer Token

In [38]:
from azure.identity import DeviceCodeCredential
 
credential = DeviceCodeCredential(tenant_id="31a2fec0-266b-4c67-b56e-2796d8f59c36", client_id ="00c104af-b0ae-4557-9787-6e6cfced741e")

In [ ]:
# Get the token
token = credential.get_token("https://cognitiveservices.azure.com/.default")

In [41]:
# Print the token value
print(token.token)
import pickle

pickle.dump(token.token, open("azuretoken", 'wb'))

eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkpETmFfNGk0cjdGZ2lnTDNzSElsSTN4Vi1JVSIsImtpZCI6IkpETmFfNGk0cjdGZ2lnTDNzSElsSTN4Vi1JVSJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzMxYTJmZWMwLTI2NmItNGM2Ny1iNTZlLTI3OTZkOGY1OWMzNi8iLCJpYXQiOjE3NDE4OTYwNjcsIm5iZiI6MTc0MTg5NjA2NywiZXhwIjoxNzQxOTAwOTg0LCJhY3IiOiIxIiwiYWlvIjoiQVhRQWkvOFpBQUFBMG8vTmZIajk4TkoxcDNhbHRVd3RLV2NGYkdZbTdIWDc5REpwUWc4VjN0eC94eHJkczJya1EvNE5qUWdnSk1NTkQ0R0psT2dJMGxlMEh0anNvK1M5QnZTVEQ5aVUvbGdmUTBwUGFQV2Q4VXdZM1lxdDlJcWM0WmY5WWhSZGZleFo4R1FhUkhaVHpnc3N6K2JiUWVMbWlnPT0iLCJhbXIiOlsibWZhIl0sImFwcGlkIjoiMDBjMTA0YWYtYjBhZS00NTU3LTk3ODctNmU2Y2ZjZWQ3NDFlIiwiYXBwaWRhY3IiOiIwIiwiZmFtaWx5X25hbWUiOiJCdXJucyIsImdpdmVuX25hbWUiOiJBbmRyZXciLCJncm91cHMiOlsiZTJhNTZjMDEtYzkwZC00ODdhLTgyOWEtNzUzYzZmMmMxYTA1IiwiNzdkNjI1MDMtYTRiNS00ZDUwLTg0ZTEtZDc3NDM5N2IwNGJiIiwiNjNjODNhMDMtNDRlOS00NzYzLWIzNDUtZWE3OWUxZDI5YWVlIiwiMzcxMjM0MDQtM2I4Ny00OTI2LThkMTQtMGZmZDg1YmVkNjkzIiwiMDI3NmI5MDUtNDExYy0


**Explanation:**

- **Model**: We use the `gpt-35-turbo-16k` model for generating responses.
- **Extra Headers**: The `APIM subscription key` is provided as an extra header for authentication.

#### Output

Running the above code will provide a response from the model, related to financial inclusion in the context of global development.

##### Troubleshooting Tips:

1. **Authentication Issues**:
   - Ensure that Azure AD login is successful when prompted.
   
2. **APIM Key Issues**:
   - Ensure that a valid APIM subscription key is used.
   - Check that the key is associated with the World Bank’s APIM service.

3. **Response Issues**:
   - If the model fails to generate a response, ensure that the model name and deployment settings are correct.
   - Adjust the prompt for better response quality.

#### Conclusion

This notebook demonstrates how to use Azure OpenAI with Azure AD authentication to generate chat completions tailored to World Bank projects. Modify the prompt and model parameters as needed for different use cases.

---------------------------------------------Happy Coding--------------------------------------------------